In [1]:
#!pip install xgboost
import os
import sys
import warnings

PROJ_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))

sys.path.append(PROJ_PATH) 


import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import itertools
from pandas.api.types import CategoricalDtype

from datetime import date 
from datetime import datetime, timedelta 
import datetime as DT 
import time

import xgboost as xgb 

import helper_functions as hfs 
from scipy.stats import linregress 
import math
from sklearn.decomposition import PCA

from sklearn import metrics
from sklearn.metrics import confusion_matrix

warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)  # to see all columns of dataframe
pd.set_option("display.max_rows", None)  # to see all rows of dataframe
pd.set_option("display.max_colwidth", None)  # to see the full (un-truncated) dataframe field/cell value 

In [2]:
import configparser

env_config = configparser.ConfigParser()
env_config.read(f'{PROJ_PATH}/kye/env_config.ini')

# <<<<<<<<<<<<<<< >>>>>>>>>>>>>>>>>>>>>>> #

SPLIT_DATE = pd.to_datetime(env_config['ENV_VARS']['SPLIT_DATE'])
ML_DATA_FOLDER_PATH = env_config['LOCAL_PATHS']['ML_DATA_FOLDER_PATH']
PROCESSING_TEMP_FOLDER_PATH = env_config['LOCAL_PATHS']['PROCESSING_TEMP_FOLDER_PATH'] 

OUTPUTS_FOLDER_PATH = env_config['LOCAL_PATHS']['OUTPUT_FOLDER_PATH'] + 'D1/'

#@@@@@@ **********************<<<<<<<<<<<<#

print("SPLIT_DATE: ", SPLIT_DATE)

TODAY = str(date.today())
FROM_DATE = pd.to_datetime(date.today(), format='%Y-%m-%d') - DT.timedelta(days=((365*4) + 60)) # "2016-01-01"
TO_DATE = str(date.today())
TWO_YEARS_BEFORE_FROM_DATE = pd.to_datetime(FROM_DATE, format='%Y-%m-%d') - DT.timedelta(days=730)
FROM_DATE 

SPLIT_DATE:  2023-08-09 00:00:00


Timestamp('2019-06-11 00:00:00')

In [3]:
analytics_rec_dict = {}
model_feed_dict = {}

#SPLIT_DATE = pd.to_datetime("2022-06-07")
#SPLIT_DATE


In [4]:
def get_last_N_days(N_days):
    unique_dates_list = [x for x in analytics_df['Date'].unique()]
    split_date_index = unique_dates_list.index(SPLIT_DATE)
    N_days_dates = unique_dates_list[split_date_index-(N_days-1):split_date_index+1]
    return N_days_dates 

In [6]:
files_in_ML_DATA = os.listdir(f"{PROJ_PATH}/{ML_DATA_FOLDER_PATH}") 
files_in_ML_DATA


['all_stocks_ohlcv.pkl',
 'stocks_current_fundamentals_1340_2023-08-09.pkl',
 'indices_daily_ohlcv_2023-08-09.pkl',
 '.ipynb_checkpoints']

In [7]:
files_in_PROCESSING_TEMP = os.listdir(f"{PROJ_PATH}/{PROCESSING_TEMP_FOLDER_PATH}") 
files_in_PROCESSING_TEMP


['RIPOs_df.pkl',
 'analytics_feature_ind.pkl',
 'indices_daily_ohlcv_2023-08-09.pkl',
 '.ipynb_checkpoints',
 'OHLCV_all_stocks_ohlcv_data_prep.pkl']

In [8]:
analytics_df = pd.read_pickle( f"{PROJ_PATH}/{PROCESSING_TEMP_FOLDER_PATH}{[x for x in files_in_PROCESSING_TEMP if 'analytics_feature_ind' in x][0]}") 
analytics_df = analytics_df[-analytics_df['Ticker'].isin(['BIPH', 'T-P-C','DNAD', 'GRPH', 'SPIR', 'ZWZZT'
                                                                                            ])].reset_index(drop=True) 
print("#Tickers: ", analytics_df['Ticker'].nunique())
print(analytics_df.shape)
analytics_df.tail(2) 

#Tickers:  1340
(334383, 309)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [9]:
N_days = 4
unique_dates_list = [x for x in analytics_df['Date'].unique()]
split_date_index = unique_dates_list.index(SPLIT_DATE)
N_days_dates = unique_dates_list[split_date_index-(N_days-1):split_date_index+1]
N_days_dates 

analytics_df['Vol_price'] = analytics_df['OHLC_price'] * analytics_df['Adj_Vol'] 


eligible_tickers = analytics_df["Ticker"][ ((analytics_df['Vol_price']>=1000000) & 
                                            (analytics_df['Adj_Vol']>=500000)
                                           ) &
                                          (analytics_df['Date'].isin([x for x in N_days_dates]))
                                         ].unique()

analytics_df = analytics_df[analytics_df['Ticker'].isin(eligible_tickers)].reset_index(drop=True)

len(eligible_tickers) 

923

In [11]:
stocks_current_fundamentals = pd.read_pickle(f"{PROJ_PATH}/{ML_DATA_FOLDER_PATH}/{[x for x in files_in_ML_DATA if 'stocks_current_fundamentals' in x][0]}") 
stocks_current_fundamentals.head(5)

,Ticker,General__AddressData.Country,General__Sector,Highlights__EPSEstimateCurrentQuarter,Highlights__ReturnOnEquityTTM,Valuation__PriceBookMRQ,SharesStats__PercentInsiders,SharesStats__PercentInstitutions,SharesStats__SharesFloat,SharesStats__PercentRetailers
0,WMB,United States,Energy,0.38,0.2037,3.6408,0.356,87.527,1211.263836,12.117
1,FUTU,Hong Kong,Financial Services,0.82,0.1667,3.9834,16.407,39.594,56.984457,43.999
2,DRUG,Other,Healthcare,0.00,-1.0154,2.7138,23.666,3.006,2.993403,73.328
3,KZR,United States,Healthcare,-0.29,-0.3070,1.6878,18.627,86.686,31.949684,-5.313
4,GFS,United States,Technology,0.49,0.1633,3.9494,0.002,102.391,82.518699,-2.393


In [12]:
#stocks_current_fundamentals = pd.read_pickle(f"{PROJ_PATH}{ML_DATA_FOLDER_PATH}{[x for x in files_in_ML_DATA if 'stocks_current_fundamentals' in x][0]}") 

#stocks_current_fundamentals = stocks_current_fundamentals[['Ticker', 'General__Sector', 'General__GicIndustry', 'General__GicSubIndustry']]

stocks_current_fundamentals = stocks_current_fundamentals[stocks_current_fundamentals['Ticker'].isin(analytics_df['Ticker'].unique())].reset_index(drop=True)
print(stocks_current_fundamentals.shape) 
stocks_current_fundamentals.head(3) 


(923, 10)


,Ticker,General__AddressData.Country,General__Sector,Highlights__EPSEstimateCurrentQuarter,Highlights__ReturnOnEquityTTM,Valuation__PriceBookMRQ,SharesStats__PercentInsiders,SharesStats__PercentInstitutions,SharesStats__SharesFloat,SharesStats__PercentRetailers
0,WMB,United States,Energy,0.38,0.2037,3.6408,0.356,87.527,1211.263836,12.117
1,FUTU,Hong Kong,Financial Services,0.82,0.1667,3.9834,16.407,39.594,56.984457,43.999
2,DRUG,Other,Healthcare,0.00,-1.0154,2.7138,23.666,3.006,2.993403,73.328


In [13]:
print(analytics_df.shape)
analytics_df = pd.merge(analytics_df, 
                        stocks_current_fundamentals[['Ticker','SharesStats__PercentInsiders', 'SharesStats__PercentInstitutions']], 
                                    how='left', on = "Ticker") 
analytics_df.shape

(231767, 310)


(231767, 312)

In [14]:
analytics_df['Tomo_Close'] = analytics_df.groupby("Ticker")['Adj_Close'].shift(-1)
analytics_df['Tomo_Close_Hike_per'] = 100 * (analytics_df['Tomo_Close'] - analytics_df['Adj_Close']) / analytics_df['Adj_Close']

analytics_df['Tomo_High'] = analytics_df.groupby("Ticker")['Adj_High'].shift(-1)
analytics_df['Tomo_High_Hike_per'] = 100 * (analytics_df['Tomo_High'] - analytics_df['Adj_Close']) / analytics_df['Adj_Close']

analytics_df['Tomo_Vol'] = analytics_df.groupby("Ticker")['Adj_Vol'].shift(-1)
analytics_df['Tomo_Vol_Hike_per'] = 100 * (analytics_df['Tomo_Vol'] - analytics_df['Adj_Vol']) / analytics_df['Adj_Vol']



In [15]:
analytics_df.tail(2)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [16]:
for i in [1,2,3]:
    analytics_df[f'EMA_5_{i}d'] = analytics_df.groupby("Ticker")['EMA_5'].shift(i)
    analytics_df[f'EMA_20_{i}d'] = analytics_df.groupby("Ticker")['EMA_20'].shift(i)


In [17]:
for i in [0,1,2,3,4,5,6,7,8,9,10]:
    analytics_df[f'Adj_Vol_before_{i}d'] = analytics_df.groupby("Ticker")['Adj_Vol'].shift(i)

for i in [1,2,3,4,5]:
    analytics_df[f'Vol_change_per_{i}d'] = (analytics_df[f'Adj_Vol_before_{i-1}d'] - analytics_df[f'Adj_Vol_before_{i}d']) / analytics_df[f'Adj_Vol_before_{i}d']
    analytics_df[f'Vol_change_per_{i}d'].fillna(0,inplace=True)
    


In [18]:
analytics_df.tail(2)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [19]:
for i in [0,1,2,3,4,5,6]:
    analytics_df[f'Adj_Close_before_{i}d'] = analytics_df.groupby("Ticker")['Adj_Close'].shift(i)

for i in [1,2,3,4,5,6]:
    analytics_df[f'Adj_Close_change_per_{i}d'] = (analytics_df[f'Adj_Close_before_{i-1}d'] - analytics_df[f'Adj_Close_before_{i}d']) / analytics_df[f'Adj_Close_before_{i}d']
    analytics_df[f'Adj_Close_change_per_{i}d'].fillna(0,inplace=True)
print(analytics_df.shape)
analytics_df.tail(2) 

for i in [0,1,2,3,4,5,6,7,8,9,10]:
    analytics_df[f'Adj_High_before_{i}d'] = analytics_df.groupby("Ticker")['Adj_High'].shift(i)

for i in [1,2,3,4,5,6,7,8,9,10]:
    analytics_df[f'Adj_High_change_per_{i}d'] = (analytics_df[f'Adj_High_before_{i-1}d'] - analytics_df[f'Adj_High_before_{i}d']) / analytics_df[f'Adj_High_before_{i}d']
    analytics_df[f'Adj_High_change_per_{i}d'].fillna(0,inplace=True)
print(analytics_df.shape)
analytics_df.tail(2) 


for i in [0,1,2,3,4,5,6,7,8,9,10]:
    analytics_df[f'Adj_Vol_before_{i}d'] = analytics_df.groupby("Ticker")['Adj_Vol'].shift(i)
    
    
    
for i in [0,1,2,3,4,5,6,7,8,9,10]:
    analytics_df[f'Adj_Vol_before_{i}d'] = analytics_df.groupby("Ticker")['Adj_Vol'].shift(i)
    
analytics_df['high_highest_5d'] = analytics_df['Adj_High'].rolling(5).max()
analytics_df['high_highest_10d'] = analytics_df['Adj_High'].rolling(10).max()

analytics_df['Vol_highest_5d'] = analytics_df['Adj_Vol'].rolling(10).max()
analytics_df['Vol_highest_10d'] = analytics_df['Adj_Vol'].rolling(10).max()



for i in [0,1,2,3,4,5,6]:
    analytics_df[f'Adj_High_before_{i}d'] = analytics_df.groupby("Ticker")['Adj_High'].shift(i)

for i in [1,2,3,4,5]:
    analytics_df[f'High_hike_change_per_{i}d'] = (analytics_df[f'Adj_High_before_{i-1}d'] - analytics_df[f'Adj_Close_before_{i}d']) / analytics_df[f'Adj_Close_before_{i}d']
    analytics_df[f'High_hike_change_per_{i}d'].fillna(0,inplace=True)
print(analytics_df.shape) 


(231767, 340)
(231767, 361)
(231767, 370)


In [20]:
analytics_df.head(2)

,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [21]:
analytics_df[f'Adj_Vol_1d'] = analytics_df.groupby("Ticker")['Adj_Vol'].shift(1)

for i in [3,5,20]:
    # analytics_df[f'Adj_Vol_EMA_prev_{i}d'] = analytics_df['Adj_Vol_1d'].ewm(span=i).mean() #
    analytics_df[f'Adj_Vol_EMA_prev_{i}d'] = analytics_df['Adj_Vol_1d'].rolling(i).mean().round(decimals=3)
    

In [22]:
print(analytics_df.shape)
print(analytics_df['Ticker'].nunique())
analytics_df.tail(2)

(231767, 371)
923


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [23]:
analytics_df['High_Low_diff'] = analytics_df['Adj_High'] - analytics_df['Adj_Low']
analytics_df['Close_High_diff'] = analytics_df['Adj_Close'] - analytics_df['Adj_High']
analytics_df['Close_Low_diff'] = analytics_df['Adj_Close'] - analytics_df['Adj_Low'] 

In [24]:
last_5_days_gr_df = analytics_df.groupby('Ticker').tail(5).groupby('Ticker').agg({ 'Adj_Close_change_per_1d' : 'unique' }).reset_index()
last_5_days_gr_df.columns = ['Ticker', 'last_5_days_gr_status']

last_10_days_gr_df = analytics_df.groupby('Ticker').tail(10).groupby('Ticker').agg({ 'Adj_Close_change_per_1d' : 'unique' }).reset_index()
last_10_days_gr_df.columns = ['Ticker', 'last_10_days_gr_status']


print(analytics_df.shape)
analytics_df = pd.merge(analytics_df, last_5_days_gr_df, on='Ticker')
analytics_df = pd.merge(analytics_df, last_10_days_gr_df, on='Ticker')
print(analytics_df.shape)

analytics_df['5_day_negatives'] = analytics_df['last_5_days_gr_status'].apply(lambda x: len(list(filter(lambda i: i < 0, x))) )
analytics_df['10_day_negatives'] = analytics_df['last_10_days_gr_status'].apply(lambda x: len(list(filter(lambda i: i < 0, x))) )
analytics_df.head(2) 

(231767, 372)
(231767, 374)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [25]:
analytics_df['Date'][0]

Timestamp('2022-08-04 00:00:00')

In [26]:
analytics_df['Opening_Gap_Percent'] = (analytics_df['Adj_Open'] - analytics_df['Adj_Close_before_1d']) /  analytics_df['Adj_Close_before_1d']

analytics_df['day_Open_high_per'] = analytics_df['High_Open_diff'] / analytics_df['Adj_Open']
analytics_df['day_Open_low_per'] = analytics_df['Low_Open_diff'] / analytics_df['Adj_Open']
analytics_df['day_Open_close_per'] = analytics_df['Close_Open_diff'] / analytics_df['Adj_Open']

analytics_df['EMA_prev_20d_DIV_Adj_Vol'] = analytics_df['Adj_Vol_EMA_prev_20d'] / analytics_df['Adj_Vol']
analytics_df['EMA_prev_5d_DIV_Adj_Vol'] = analytics_df['Adj_Vol_EMA_prev_5d'] / analytics_df['Adj_Vol']

analytics_df['ATR_3_DIV_Adj_Close'] = analytics_df['ATR_3'] / analytics_df['Adj_Close']
                         
analytics_df['Cam_R1_day_5P_DIV_Cam_R1_day_5'] = analytics_df['Cam_R1_day_5P'] / analytics_df['Cam_R1_day_5'] 
analytics_df['Cam_S1_day_5P_DIV_Cam_S1_day_5'] = analytics_df['Cam_S1_day_5P'] / analytics_df['Cam_S1_day_5'] 
analytics_df['Cam_R3_day_1P_DIV_Cam_R3_day_1'] = analytics_df['Cam_R3_day_1P'] / analytics_df['Cam_R3_day_1']  
analytics_df['Cam_S3_day_1P_DIV_Cam_S3_day_1'] = analytics_df['Cam_S3_day_1P'] / analytics_df['Cam_S3_day_1']  
                         
analytics_df['Adj_Close_DIV_EMA_9'] = analytics_df['Adj_Close'] / analytics_df['EMA_9'] 
analytics_df['EMA_9_DIV_MA50'] = analytics_df['EMA_9'] / analytics_df['Adj_Close_MA50'] 
analytics_df['MA50_DIV_MA150'] = analytics_df['Adj_Close_MA50'] / analytics_df['Adj_Close_MA150'] 
analytics_df['MA150_DIV_MA200'] = analytics_df['Adj_Close_MA150'] / analytics_df['Adj_Close_MA200']           
                         
analytics_df['Adj_Close_DIV_low_52_weeks'] = analytics_df['Adj_Close'] / analytics_df['Low_52_weeks']
analytics_df['Adj_Close_DIV_high_52_weeks'] = analytics_df['Adj_Close'] / analytics_df['High_52_weeks']

analytics_df['Adj_Close_DIV_MA50'] = analytics_df['Adj_Close'] / analytics_df['Adj_Close_MA50']

analytics_df['EMA_5_DIV_EMA_20'] = analytics_df['EMA_5'] / analytics_df['EMA_20']
analytics_df['EMA_5_1d_DIV_EMA_20_1d'] = analytics_df['EMA_5_1d'] / analytics_df['EMA_20_1d'] 
analytics_df['EMA_5_2d_DIV_EMA_20_2d'] = analytics_df['EMA_5_2d'] / analytics_df['EMA_20_2d'] 
analytics_df['EMA_5_3d_DIV_EMA_20_3d'] = analytics_df['EMA_5_3d'] / analytics_df['EMA_20_3d']

features_for_model = ['price_cat','Highlights__EPSEstimateCurrentQuarter', 'Highlights__ReturnOnEquityTTM',
                       'Valuation__PriceBookMRQ', 'SharesStats__PercentInsiders',
                       'SharesStats__PercentInstitutions', 'SharesStats__SharesFloat',
                       'Adj_Vol', 'Vol_Price', 'Opening_Gap_Percent', 'Adj_Close_change_per_1d', 'Adj_Close_change_per_2d', 'Adj_Close_change_per_3d',
                       'Adj_Close_change_per_5d', 'Adj_Close_change_per_10d', 'Adj_Close_change_per_20d', 
                       '5_day_negatives', '10_day_negatives', 'straight_down_per', 'straight_up_per', 'straight_up_close_per', 
                       'SP_status2d',  'EMA_prev_20d_DIV_Adj_Vol', 'EMA_prev_5d_DIV_Adj_Vol', 'ATR_3','ATR_3_DIV_Adj_Close', 
                       'Cam_R1_day_5P_DIV_Cam_R1_day_5', 'Cam_S1_day_5P_DIV_Cam_S1_day_5', 'Cam_R3_day_1P_DIV_Cam_R3_day_1', 
                       'Cam_S3_day_1P_DIV_Cam_S3_day_1', 'Adj_Close_DIV_EMA_9', 
                       'EMA_9_DIV_MA50', 'MA50_DIV_MA150', 'MA150_DIV_MA200', 'Adj_Close_DIV_low_52_weeks', 'Adj_Close_DIV_high_52_weeks', 
                       'Adj_Close_DIV_MA50', 'EMA_5_DIV_EMA_20', 'EMA_5_1d_DIV_EMA_20_1d', 'EMA_5_2d_DIV_EMA_20_2d', 
                       'EMA_5_2d_DIV_EMA_20_2d', 'EMA_5_3d_DIV_EMA_20_3d' ]

key_action_day_vars = ['price_cat', 'SharesStats__SharesFloat', 'Adj_Vol', 'Vol_Price', 'Opening_Gap_Percent', 'Adj_Close_change_per_1d',
                       'EMA_prev_20d_DIV_Adj_Vol', 'EMA_prev_5d_DIV_Adj_Vol', 'ATR_3','ATR_3_DIV_Adj_Close', 
                       'Adj_Close_DIV_low_52_weeks', 'Adj_Close_DIV_high_52_weeks',
                       'SharesStats__PercentInsiders', 'SharesStats__PercentInstitutions', 
                       'Adj_Close_change_per_2d', 'Adj_Close_change_per_3d', 'Adj_Close_change_per_5d', 'Adj_Close_change_per_10d', 'Adj_Close_change_per_20d', 
                       '5_day_negatives', '10_day_negatives', 'SP_status2d',
                       'Cam_R1_day_5P_DIV_Cam_R1_day_5', 'Cam_S1_day_5P_DIV_Cam_S1_day_5', 'Cam_R3_day_1P_DIV_Cam_R3_day_1', 
                       'Cam_S3_day_1P_DIV_Cam_S3_day_1' ]

key_prev_action_day_vars = ['price_cat', 'SharesStats__SharesFloat', 'Adj_Vol', 'Vol_Price', 'Opening_Gap_Percent', 'Adj_Close_change_per_1d',
                           'EMA_prev_20d_DIV_Adj_Vol', 'EMA_prev_5d_DIV_Adj_Vol', 'ATR_3','ATR_3_DIV_Adj_Close', 
                           'Adj_Close_DIV_low_52_weeks', 'Adj_Close_DIV_high_52_weeks',
                           'Adj_Close_DIV_MA50', 'Adj_Close_DIV_EMA_9', 'EMA_9_DIV_MA50', 'MA50_DIV_MA150', 'MA150_DIV_MA200', 
                           'SharesStats__PercentInsiders', 'SharesStats__PercentInstitutions', 
                           'Adj_Close_change_per_2d', 'Adj_Close_change_per_3d',
                           'Adj_Close_change_per_5d', 'Adj_Close_change_per_10d', 'Adj_Close_change_per_20d', 
                           '5_day_negatives', '10_day_negatives', 'straight_down_per', 'straight_up_per', 'straight_up_close_per', 'SP_status2d',
                           'Cam_R1_day_5P_DIV_Cam_R1_day_5', 'Cam_S1_day_5P_DIV_Cam_S1_day_5', 'Cam_R3_day_1P_DIV_Cam_R3_day_1', 
                           'Cam_S3_day_1P_DIV_Cam_S3_day_1'] 

In [27]:
price_cat_list = []

price_cat_list = [0, 0.4, 0.8, 1.2, 1.6] + \
                 [x for x in range(2,20)] + [x for x in range(20,50,3)] + \
                 [x for x in range(50,100,5)] + [x for x in range(100,240,10)]

print(len(price_cat_list))

analytics_df["price_cat_smaller"] = pd.cut(x= analytics_df.Adj_Close,
                                           bins=price_cat_list,
                                           include_lowest=True,
                                           labels=price_cat_list[1:])

analytics_df["price_cat_smaller"] = np.where( pd.isnull(analytics_df["price_cat_smaller"]),
                                              240,
                                              analytics_df["price_cat_smaller"]
                                            )


57


In [28]:
top_15_cats = analytics_df["price_cat_smaller"].value_counts().head(20).index
print(top_15_cats.min(), top_15_cats.max())

1.2 38.0


In [29]:
analytics_df[ (analytics_df['Adj_Close']>=1.5) &
              (analytics_df['Adj_Close']<=28) &
              (analytics_df['Date'].isin(get_last_N_days(2))) 
            ]['Ticker'].nunique()
             

585

In [30]:
analytics_df.head(2)

,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

# Best Runners/Longs

In [31]:
analytics_df_5d = analytics_df[(analytics_df['Date'].isin(get_last_N_days(5))) &
                               (analytics_df['Adj_Vol']>=2000000)
                              ].reset_index(drop=True). \
                  sort_values(by=['Date', 'day_Open_high_per'], ascending=[True, False]).reset_index(drop=True)

analytics_df_5d = analytics_df_5d.groupby(['Date','price_cat']).head(12)
print(analytics_df_5d.shape)
analytics_df_5d['Adj_Vol'] = analytics_df_5d['Adj_Vol'] / 1000000
analytics_df_5d['Vol_Price'] = analytics_df_5d['Vol_Price'] / 1000000
#analytics_df_5d['SharesStats__SharesFloat'] = analytics_df_5d['SharesStats__SharesFloat'] / 1000000

action_day_analytics = analytics_df_5d.groupby(['price_cat']).agg({
                                        'High_Low_diff' : ['min','median','mean','max'], 
                                        'Close_High_diff' : ['min','median','mean','max'], 
                                        'Close_Low_diff' : ['min','median','mean','max'], 
                                        'SharesStats__SharesFloat' : ['min','median','mean','max'], 
                                        'ADR_percent_5d': ['min','median','mean','max'], 
                                        'Adj_Vol' : ['min','median','mean','max'], 
                                        'Opening_Gap_Percent' : ['min','median','mean','max'], 
                                        'day_Open_high_per' : ['min','median','mean','max'],
                                        'day_Open_low_per' :  ['min','median','mean','max'],
                                        'day_Open_close_per': ['min','median','mean','max'],
                                        'Adj_Close_change_per_1d' : ['min','median','mean','max'],
                                        'EMA_prev_20d_DIV_Adj_Vol' : ['min','median','mean','max'], 
                                        'Adj_Close_DIV_low_52_weeks' : ['min','median','mean','max'], 
                                        'Adj_Close_DIV_high_52_weeks' : ['min','median','mean','max'],
                                        'SharesStats__PercentInsiders' : ['min','median','mean','max'], 
                                        'SharesStats__PercentInstitutions' : ['min','median','mean','max'], 
                                        '5_day_negatives' : ['min','median','mean','max'], 
                                        '10_day_negatives' : ['min','median','mean','max']
                                    }).reset_index().T.round(3)

action_day_analytics.columns = list(action_day_analytics.iloc[0:1,:].values[0]) 
action_day_analytics.drop(['price_cat'], inplace=True) 

action_day_analytics = action_day_analytics[['LT_2', 'GT_2_LT_5',  'GT_5_LT_20',  'GT_20']] 


(240, 382)


In [32]:
price_risk_line_df = analytics_df_5d.groupby(['price_cat_smaller']).agg({
                                        'High_Low_diff' : ['min','median','mean','max'] }).reset_index().round(3)

price_risk_line_df.columns = ['price_cat_smaller','min','median','mean','max'] 


## Action day analytics - Last 5 day Runners

In [35]:
action_day_analytics 

LT_2    GT_2_LT_5   GT_5_LT_20  \
High_Low_diff                    min          0.009         0.13         0.27   
                                 median        0.17         0.55         1.17   
                                 mean      0.272883     0.684683     1.395317   
                                 max           1.32         2.98         4.57   
Close_High_diff                  min           -1.2        -2.27    -2.959999   
                                 median       -0.04         -0.2    -0.287499   
                                 mean      -0.15195      -0.3486      -0.5238   
                                 max            0.0          0.0          0.0   
Close_Low_diff                   min            0.0          0.0         0.01   
                                 median       0.065       0.2875        0.605   
                                 mean      0.120933     0.336083     0.871517   
                                 max            0.8         1.34         4.41   
SharesStats__SharesFloat         min            0.0     1.435861          0.0   
                                 median     63.6446     86.67033   103.128774   
                                 mean    121.977109   169.817175   221.148682   
                                 max      987.70549  1218.196584  2207.514677   
ADR_percent_5d                   min          6.861         4.03        3.315   
                                 median   19.311001      10.9125       7.8585   
                                 mean     24.309549    19.433199    10.364117   
                                 max     120.771004   100.403999    49.185001   
Adj_Vol                          min       2.031691       2.0525       2.0493   
                                 median       8.003     7.122053     8.031582   
                                 mean     57.893962    21.264131    13.639145   
                                 max     918.234624   171.333792        89.78   
Opening_Gap_Percent              min      -0.690722    -0.439252    -0.326829   
                                 median   -0.002262     0.005575     0.006439   
                                 mean      0.049445     0.015519     0.032432   
                                 max       1.660194     0.652174     0.372573   
day_Open_high_per                min       0.070575     0.028428     0.042609   
                                 median    0.150221     0.128218     0.109851   
                                 mean      0.244258     0.189766       0.1432   
                                 max       1.019417     1.602151        0.878   
day_Open_low_per                 min      -0.285714    -0.302632    -0.125616   
                                 median   -0.034551     -0.02259    -0.015753   
                                 mean     -0.057048    -0.050974    -0.021688   
                                 max            0.0          0.0          0.0   
day_Open_close_per               min      -0.285714    -0.216165    -0.109606   
                                 median    0.075851     0.053137     0.062512   
                                 mean      0.082909     0.072562     0.082011   
                                 max       0.563786      0.72043        0.562   
Adj_Close_change_per_1d          min      -0.626804    -0.456075    -0.223415   
                                 median    0.072751     0.072652     0.076195   
                                 mean      0.147833     0.085528     0.121409   
                                 max        2.84466     0.658031     1.039165   
EMA_prev_20d_DIV_Adj_Vol         min       0.003781     0.008286       0.0023   
                                 median    0.561454     0.465553     0.390061   
                                 mean      0.772812     0.520645     0.487914   
                                 max       3.509001     1.884885     2.149763   
Adj_Close_DIV_low_52_weeks       min       1.048951     1.017964     1.051457 

In [36]:
last_30_dates = get_last_N_days(30)
last_30_dates[0]

numpy.datetime64('2023-06-28T00:00:00.000000000')

In [37]:
analytics_df['Date'][0]

Timestamp('2022-08-04 00:00:00')

In [38]:
analytics_df_5d['Date_lag1'] = analytics_df_5d['Date'].apply(lambda x: last_30_dates[last_30_dates.index(x) - 1])

analytics_df_5dPrev = pd.merge(analytics_df_5d[['Date_lag1','Ticker']],
                               analytics_df,
                               how='left',
                               left_on = ['Date_lag1', 'Ticker'],
                               right_on = ['Date', 'Ticker']
                              )

analytics_df_5dPrev['Adj_Vol'] = analytics_df_5dPrev['Adj_Vol'] / 1000000
analytics_df_5dPrev['Vol_Price'] = analytics_df_5dPrev['Vol_Price'] / 1000000
analytics_df_5dPrev['SharesStats__SharesFloat'] = analytics_df_5dPrev['SharesStats__SharesFloat'] / 1000000


OneDay_prev_action_analytics = analytics_df_5dPrev.groupby(['price_cat']).agg({
                                        'SharesStats__SharesFloat' : ['min','median','mean','max'], 
                                        'ADR_percent_5d': ['min','median','mean','max'], 
                                        'Adj_Vol' : ['min','median','mean','max'], 
                                        'Vol_Price' : ['min','median','mean','max'], 
                                        'Opening_Gap_Percent' : ['min','median','mean','max'], 
                                        'day_Open_high_per' : ['min','median','mean','max'],
                                        'day_Open_low_per' :  ['min','median','mean','max'],
                                        'day_Open_close_per': ['min','median','mean','max'],
                                        'Adj_Close_change_per_1d' : ['min','median','mean','max'],
                                        'EMA_prev_20d_DIV_Adj_Vol' : ['min','median','mean','max'], 
                                        'EMA_prev_5d_DIV_Adj_Vol' : ['min','median','mean','max'], 
                                        'ATR_3' : ['min','median','mean','max'],
                                        'ATR_3_DIV_Adj_Close' : ['min','median','mean','max'], 
                                        'Adj_Close_DIV_low_52_weeks' : ['min','median','mean','max'], 
                                        'Adj_Close_DIV_high_52_weeks' : ['min','median','mean','max'],
                                        'SharesStats__PercentInsiders' : ['min','median','mean','max'], 
                                        'SharesStats__PercentInstitutions' : ['min','median','mean','max'], 
                                        'Adj_Close_change_per_2d' : ['min','median','mean','max'], 
                                        'Adj_Close_change_per_3d' : ['min','median','mean','max'], 
                                        'Adj_Close_change_per_5d' : ['min','median','mean','max'], 
                                        'Adj_Close_change_per_10d' : ['min','median','mean','max'], 
                                        'Adj_Close_change_per_20d' : ['min','median','mean','max'], 
                                        '5_day_negatives' : ['min','median','mean','max'], 
                                        '10_day_negatives' : ['min','median','mean','max']

                                    }).reset_index().T.round(3)

OneDay_prev_action_analytics.columns = list(OneDay_prev_action_analytics.iloc[0:1,:].values[0])
OneDay_prev_action_analytics.drop(['price_cat'], inplace=True)

OneDay_prev_action_analytics = OneDay_prev_action_analytics[['LT_2', 'GT_2_LT_5',  'GT_5_LT_20',  'GT_20']]


## 1 Day Prior to action analytics - Last 5 day Runners

In [39]:
## Tickers that grew by 80%, 100% or 200%+ in last 1, 2 or 3 days
best_runners_recent = analytics_df[( (analytics_df['Adj_Close_change_per_1d']>0.5) |
                               (analytics_df['Adj_Close_change_per_2d']>0.7) | 
                               (analytics_df['Adj_Close_change_per_3d']>0.85) |
                               (analytics_df['Adj_Close_change_per_5d']>1) |
                               (analytics_df['Adj_Close_change_per_10d']>1.5) |
                               (analytics_df['Adj_Close_change_per_20d']>3) 
                             ) &
                             (analytics_df['Adj_Vol']>=2000000) &
                             # ( analytics_df['Adj_Vol'] >= 3 * analytics_df['Adj_Vol_EMA_prev_20d'] ) &
                             (analytics_df['Date'].isin(get_last_N_days(3))) 
                            ]['Ticker'].unique() 


best_runners_recent = [x for x in best_runners_recent]

best_runners_recent_df = analytics_df[ (analytics_df['Ticker'].isin(best_runners_recent) ) &
                                      (analytics_df['Date'].isin(get_last_N_days(20))) 
                                     
                                    ].reset_index(drop=True)


analytics_rec_dict['best_runners_recent'] = best_runners_recent


In [40]:
analytics_df[analytics_df['Ticker']=='OSCR'].tail(3) 


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

## Best Runners list for Everyday study

In [41]:
analytics_df_1d = analytics_df[(analytics_df['Date'].isin(get_last_N_days(1))) &
                               (analytics_df['Adj_Vol']>=2000000)
                              ].reset_index(drop=True). \
                  sort_values(by=['Date', 'day_Open_high_per'], ascending=[True, False]).reset_index(drop=True)

analytics_df_1d = analytics_df_1d.groupby(['Date','price_cat']).head(4)
print(analytics_df_1d.shape)
analytics_df_1d['Adj_Vol'] = analytics_df_1d['Adj_Vol'] / 1000000
analytics_df_1d['Vol_Price'] = analytics_df_1d['Vol_Price'] / 1000000
#analytics_df_1d['SharesStats__SharesFloat'] = analytics_df_1d['SharesStats__SharesFloat'] / 1000000
analytics_df_1d['day_Open_high_per'] = analytics_df_1d['day_Open_high_per'] * 100

analytics_df_1d[['Date','price_cat','Ticker', 'Adj_Close','day_Open_high_per', 'ADR_percent_5d',
                 'High_Open_diff','General__Sector']].sort_values(
                by=['Date', 'price_cat','day_Open_high_per'], ascending=[True, True, False]).reset_index(drop=True)

# 'ADR_percent_5d': ['min','median','mean','max'], 

(16, 382)


,Date,price_cat,Ticker,Adj_Close,day_Open_high_per,ADR_percent_5d,High_Open_diff,General__Sector
0,2023-08-09,GT_20,BROS,32.990002,18.307486,7.136000,5.430,Consumer Cyclical
1,2023-08-09,GT_20,ARRY,22.180000,8.842592,4.745000,1.910,Technology
2,2023-08-09,GT_20,RIVN,22.350000,8.200168,8.921000,1.950,Consumer Cyclical
3,2023-08-09,GT_20,TWNK,23.490000,6.422433,3.519000,1.420,Consumer Defensive
4,2023-08-09,GT_2_LT_5,BNTC,3.200000,160.215057,40.723999,2.980,Healthcare
5,2023-08-09,GT_2_LT_5,IDAI,2.450000,74.846626,35.224998,1.220,Technology
6,2023-08-09,GT_2_LT_5,TDUP,4.060000,16.193182,10.981000,0.570,Consumer Cyclical
7,2023-08-09,GT_2_LT_5,CLEU,2.560000,15.765765,12.445000,0.350,Consumer Defensive
8,2023-08-09,GT_5_LT_20,TNGX,7.810000,87.799995,24.513000,4.390,Healthcare
9,2023-08-09,GT_5_LT_20,LPRO,7.960000,20.144928,7.949000,1.390,Financial Services


## Industry best_runners_recent 

In [42]:
best_runners_recent_df.groupby(['General__Sector']
                             ).agg({'Ticker' : ['unique', 'nunique']})


Ticker        
                                                      unique nunique
General__Sector                                                     
Basic Materials                                       [SVFD]       1
Communication Services                                 [TDS]       1
Consumer Cyclical                                      [TUP]       1
Consumer Defensive                                      [MF]       1
Healthcare              [ADMP, BNTC, NKTR, TNGX, TTOO, VVOS]       6
Industrials                                           [YELL]       1
Technology                                      [DTSS, IDAI]       2

# Best Shorts

In [43]:
analytics_df_5d = analytics_df[(analytics_df['Date'].isin(get_last_N_days(5))) &
                               (analytics_df['Adj_Vol']>=2000000)
                              ].reset_index(drop=True). \
                  sort_values(by=['Date', 'day_Open_low_per'], ascending=[True, True]).reset_index(drop=True)

analytics_df_5d = analytics_df_5d.groupby(['Date','price_cat']).head(5)
print(analytics_df_5d.shape)
analytics_df_5d['Adj_Vol'] = analytics_df_5d['Adj_Vol'] / 1000000
analytics_df_5d['Vol_Price'] = analytics_df_5d['Vol_Price'] / 1000000
#analytics_df_5d['SharesStats__SharesFloat'] = analytics_df_5d['SharesStats__SharesFloat'] / 1000000

action_day_analytics = analytics_df_5d.groupby(['price_cat']).agg({
                                        'SharesStats__SharesFloat' : ['min','median','mean','max'], 
                                        'ADR_percent_5d': ['min','median','mean','max'], 
                                        'Adj_Vol' : ['min','median','mean','max'], 
                                        'Opening_Gap_Percent' : ['min','median','mean','max'], 
                                        'day_Open_high_per' : ['min','median','mean','max'],
                                        'day_Open_low_per' :  ['min','median','mean','max'],
                                        'day_Open_close_per': ['min','median','mean','max'],
                                        'Adj_Close_change_per_1d' : ['min','median','mean','max'],
                                        'EMA_prev_20d_DIV_Adj_Vol' : ['min','median','mean','max'], 
                                        'Adj_Close_DIV_low_52_weeks' : ['min','median','mean','max'], 
                                        'Adj_Close_DIV_high_52_weeks' : ['min','median','mean','max'],
                                        'SharesStats__PercentInsiders' : ['min','median','mean','max'], 
                                        'SharesStats__PercentInstitutions' : ['min','median','mean','max'], 
                                        '5_day_negatives' : ['min','median','mean','max'], 
                                        '10_day_negatives' : ['min','median','mean','max']
                                    }).reset_index().T.round(3)

action_day_analytics.columns = list(action_day_analytics.iloc[0:1,:].values[0])
action_day_analytics.drop(['price_cat'], inplace=True)

action_day_analytics = action_day_analytics[['LT_2', 'GT_2_LT_5',  'GT_5_LT_20',  'GT_20']] 


(100, 382)


## Action day analytics - Recent Dumps

In [44]:
action_day_analytics 

LT_2   GT_2_LT_5   GT_5_LT_20  \
SharesStats__SharesFloat         min        4.173237    1.940648    25.463922   
                                 median    81.406104   58.144759   160.402507   
                                 mean     165.787655   135.29578    229.97981   
                                 max     2053.617697   595.42746  1506.177633   
ADR_percent_5d                   min           8.812       7.572         4.19   
                                 median       19.632      14.613        8.857   
                                 mean       26.53076   24.210239      9.05744   
                                 max       58.506001  100.403999    27.492001   
Adj_Vol                          min        2.895275      2.5075     4.190903   
                                 median    20.429472    7.341995      14.2769   
                                 mean      45.099582   25.082982    15.069368   
                                 max        198.5144  160.603904      36.8571   
Opening_Gap_Percent              min        -0.69021   -0.128959    -0.284998   
                                 median          0.0    0.003165      0.00115   
                                 mean       0.033764    0.045575     0.003756   
                                 max        1.030651    0.652174     0.306082   
day_Open_high_per                min             0.0         0.0          0.0   
                                 median     0.013333    0.023529     0.003466   
                                 mean       0.031351    0.074176     0.011947   
                                 max        0.218487    0.358824     0.086207   
day_Open_low_per                 min       -0.656885   -0.302632    -0.264844   
                                 median        -0.25   -0.167702    -0.137864   
                                 mean      -0.266242    -0.17599    -0.146046   
                                 max       -0.137931   -0.083538    -0.086081   
day_Open_close_per               min       -0.623025   -0.216165    -0.227144   
                                 median    -0.210526   -0.127737    -0.109606   
                                 mean      -0.224177   -0.090814    -0.118897   
                                 max       -0.080508    0.212598    -0.024272   
Adj_Close_change_per_1d          min       -0.883217   -0.263267    -0.394253   
                                 median    -0.186441   -0.044226    -0.104083   
                                 mean      -0.184839   -0.052994    -0.117084   
                                 max        0.532567    0.295031     0.042871   
EMA_prev_20d_DIV_Adj_Vol         min         0.01043    0.009754     0.052507   
                                 median     0.265132    0.459481     0.315213   
                                 mean       0.543885    0.651518     0.518686   
                                 max        3.481041    3.482545     2.697739   
Adj_Close_DIV_low_52_weeks       min             1.0     1.01083     1.005764   
                                 median      1.47205    2.130769     1.606138   
                                 mean       2.217145    3.239497     2.191979   
                                 max         6.26556    8.372092        5.125   
Adj_Close_DIV_high_52_weeks      min        0.016333    0.175579     0.114334   
                                 median     0.123457    0.379447     0.590855   
                                 mean        0.17577    0.415453     0.568137   
                                 max        0.820652    0.990291     0.850888   
SharesStats__PercentInsiders     min             0.0         0.0        0.084   
                                 median        5.863       3.503        2.747   
                                 mean       17.22576    10.12792      7.37328   
                                 max          63.113      68.389       44.064   
SharesStats__PercentInstitutions min           0.886       0.324        26.18  

In [45]:
analytics_df_5d.head(2) 

,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [46]:
analytics_df_5d['Date'][0] in sorted(analytics_df['Date'].unique())[-30:]

True

In [47]:
last_30_dates = sorted(analytics_df['Date'].unique())[-30:]
last_30_dates[0]

numpy.datetime64('2023-06-28T00:00:00.000000000')

In [48]:
analytics_df_5d['Date_lag1'] = analytics_df_5d['Date'].apply(lambda x: last_30_dates[last_30_dates.index(x) - 1])

analytics_df_5dPrev = pd.merge(analytics_df_5d[['Date_lag1','Ticker']],
                               analytics_df,
                               how='left',
                               left_on = ['Date_lag1', 'Ticker'],
                               right_on = ['Date', 'Ticker']
                              )


In [49]:
analytics_df_5d['Date_lag1'] = analytics_df_5d['Date'].apply(lambda x: last_30_dates[last_30_dates.index(x) - 1])

analytics_df_5dPrev = pd.merge(analytics_df_5d[['Date_lag1','Ticker']],
                               analytics_df,
                               how='left',
                               left_on = ['Date_lag1', 'Ticker'],
                               right_on = ['Date', 'Ticker']
                              )

analytics_df_5dPrev['Adj_Vol'] = analytics_df_5dPrev['Adj_Vol'] / 1000000
analytics_df_5dPrev['Vol_Price'] = analytics_df_5dPrev['Vol_Price'] / 1000000
#analytics_df_5dPrev['SharesStats__SharesFloat'] = analytics_df_5dPrev['SharesStats__SharesFloat'] / 1000000


OneDay_prev_action_analytics = analytics_df_5dPrev.groupby(['price_cat']).agg({
                                        'SharesStats__SharesFloat' : ['min','median','mean','max'], 
                                        'ADR_percent_5d': ['min','median','mean','max'], 
                                        'Adj_Vol' : ['min','median','mean','max'], 
                                        'Vol_Price' : ['min','median','mean','max'], 
                                        'Opening_Gap_Percent' : ['min','median','mean','max'], 
                                        'day_Open_high_per' : ['min','median','mean','max'],
                                        'day_Open_low_per' :  ['min','median','mean','max'],
                                        'day_Open_close_per': ['min','median','mean','max'],
                                        'Adj_Close_change_per_1d' : ['min','median','mean','max'],
                                        'EMA_prev_20d_DIV_Adj_Vol' : ['min','median','mean','max'], 
                                        'EMA_prev_5d_DIV_Adj_Vol' : ['min','median','mean','max'], 
                                        'ATR_3' : ['min','median','mean','max'],
                                        'ATR_3_DIV_Adj_Close' : ['min','median','mean','max'], 
                                        'Adj_Close_DIV_low_52_weeks' : ['min','median','mean','max'], 
                                        'Adj_Close_DIV_high_52_weeks' : ['min','median','mean','max'],
                                        'SharesStats__PercentInsiders' : ['min','median','mean','max'], 
                                        'SharesStats__PercentInstitutions' : ['min','median','mean','max'], 
                                        'Adj_Close_change_per_2d' : ['min','median','mean','max'], 
                                        'Adj_Close_change_per_3d' : ['min','median','mean','max'], 
                                        'Adj_Close_change_per_5d' : ['min','median','mean','max'], 
                                        'Adj_Close_change_per_10d' : ['min','median','mean','max'], 
                                        'Adj_Close_change_per_20d' : ['min','median','mean','max'], 
                                        '5_day_negatives' : ['min','median','mean','max'], 
                                        '10_day_negatives' : ['min','median','mean','max']

                                    }).reset_index().T.round(3)

OneDay_prev_action_analytics.columns = list(OneDay_prev_action_analytics.iloc[0:1,:].values[0])
OneDay_prev_action_analytics.drop(['price_cat'], inplace=True)

OneDay_prev_action_analytics = OneDay_prev_action_analytics[['LT_2', 'GT_2_LT_5',  'GT_5_LT_20',  'GT_20']]


## 1 Day Prior to action analytics - Recent Dumps

In [50]:
OneDay_prev_action_analytics

LT_2   GT_2_LT_5   GT_5_LT_20  \
SharesStats__SharesFloat         min        6.734139    1.940648    25.463922   
                                 median    82.568653   55.138221   160.402507   
                                 mean     172.940454  128.648807   230.905464   
                                 max     2053.617697   595.42746  1506.177633   
ADR_percent_5d                   min           5.519       3.862        2.442   
                                 median      13.1845   11.922501        6.111   
                                 mean      17.620584   26.580793      7.07416   
                                 max       49.386002  100.635002       27.135   
Adj_Vol                          min          0.0543      0.0272       1.2457   
                                 median      4.33435     10.5655       6.1333   
                                 mean      37.807496   28.818398     7.890538   
                                 max      478.644896  171.333792      27.5103   
Vol_Price                        min        0.030494    0.087584     6.515011   
                                 median     4.250675   34.770664    62.835616   
                                 mean       14.49163   85.012216    87.379794   
                                 max      167.525712  428.334464   528.197792   
Opening_Gap_Percent              min       -0.623529   -0.383753    -0.046855   
                                 median          0.0   -0.001433    -0.002703   
                                 mean      -0.012868   -0.015886      0.00527   
                                 max        0.473251    0.202492     0.123879   
day_Open_high_per                min             0.0         0.0          0.0   
                                 median     0.026244    0.068258      0.01938   
                                 mean       0.056602    0.137265     0.037793   
                                 max        0.256983    0.686046     0.199803   
day_Open_low_per                 min        -0.40625   -0.279202    -0.097772   
                                 median    -0.072105    -0.04112    -0.025281   
                                 mean       -0.11536    -0.07339    -0.032767   
                                 max             0.0         0.0          0.0   
day_Open_close_per               min       -0.395833   -0.181347    -0.071518   
                                 median    -0.040072    0.003713     0.003831   
                                 mean      -0.069515    0.010845    -0.005705   
                                 max          0.1125    0.273256     0.085169   
Adj_Close_change_per_1d          min       -0.772549   -0.305322    -0.066998   
                                 median    -0.052344   -0.011756      0.00202   
                                 mean      -0.071043   -0.005398    -0.001088   
                                 max        0.440329    0.510345     0.076233   
EMA_prev_20d_DIV_Adj_Vol         min        0.011893    0.010887     0.218689   
                                 median     0.646194    0.748445     0.738508   
                                 mean       1.093422    3.586523     0.983254   
                                 max        5.455722   55.331341     4.285364   
EMA_prev_5d_DIV_Adj_Vol          min        0.023842    0.038101     0.328735   
                                 median     0.794919    0.925966     0.821431   
                                 mean       1.091399     8.31737     0.840348   
                                 max            6.03  160.839024     1.580695   
ATR_3                            min            0.01       0.106        0.157   
                                 median       0.1345      0.5175        0.588   
                                 mean       0.138333    0.750458       0.8174   
                                 max           0.395        2.14        3.257   
ATR_3_DIV_Adj_Close              min        0.051073    0.036552     0.021764  

In [51]:
## Tickers that grew by 80%, 100% or 200%+ in last 1, 2 or 3 days 
best_shorts_recent = analytics_df[( (analytics_df['Adj_Close_change_per_1d'] < -0.3) |
                                    (analytics_df['Adj_Close_change_per_2d'] < -0.5) | 
                                    (analytics_df['Adj_Close_change_per_3d'] < -0.6) |
                                    (analytics_df['Adj_Close_change_per_5d'] < -0.65 ) |
                                    (analytics_df['Adj_Close_change_per_10d'] < -0.7 ) |
                                    (analytics_df['Adj_Close_change_per_20d'] < -0.75 ) 
                                   ) &
                                     (analytics_df['Adj_Vol']>=1500000) &
                                    # ( analytics_df['Adj_Vol'] >= 2.5 * analytics_df['Adj_Vol_EMA_prev_20d'] ) &
                                     (analytics_df['Date'].isin(get_last_N_days(5))) 
                                  ]['Ticker'].unique() 
best_shorts_recent = [x for x in best_shorts_recent]

best_shorts_recent_df = analytics_df[ (analytics_df['Ticker'].isin(best_shorts_recent) ) &
                                      (analytics_df['Date'].isin(get_last_N_days(20))) 
                                     
                                    ].reset_index(drop=True)

analytics_rec_dict['best_shorts_recent'] = best_shorts_recent 

## Best Dumps list for Everyday study

In [52]:
print(f"best_shorts_recent: {len(best_shorts_recent)} : \n {best_shorts_recent}") 

best_shorts_recent: 13 : 
 ['ADMP', 'ARAV', 'ASRT', 'MRSN', 'NVEI', 'PALI', 'POL', 'PTRA', 'SAGE', 'UPST', 'VRM', 'WE', 'YELL']


## Industry best Dumps recent

In [53]:
best_shorts_recent_df.groupby(['General__Sector']
                              ).agg({'Ticker' : ['unique', 'nunique']})

Ticker        
                                                  unique nunique
General__Sector                                                 
Consumer Cyclical                             [POL, VRM]       2
Financial Services                                [UPST]       1
Healthcare          [ADMP, ARAV, ASRT, MRSN, PALI, SAGE]       6
Industrials                                 [PTRA, YELL]       2
Real Estate                                         [WE]       1
Technology                                        [NVEI]       1

In [54]:
analytics_df_1d = analytics_df[(analytics_df['Date'].isin(get_last_N_days(1))) &
                               (analytics_df['Adj_Vol']>=2000000)
                              ].reset_index(drop=True). \
                  sort_values(by=['Date', 'day_Open_low_per'], ascending=[True, True]).reset_index(drop=True)

analytics_df_1d = analytics_df_1d.groupby(['Date','price_cat']).head(4)
print(analytics_df_1d.shape)
analytics_df_1d['Adj_Vol'] = analytics_df_1d['Adj_Vol'] / 1000000
analytics_df_1d['Vol_Price'] = analytics_df_1d['Vol_Price'] / 1000000
#analytics_df_1d['SharesStats__SharesFloat'] = analytics_df_1d['SharesStats__SharesFloat'] / 1000000
analytics_df_1d['day_Open_low_per'] = analytics_df_1d['day_Open_low_per'] * 100

analytics_df_1d[['Date','price_cat','Ticker', 'Adj_Close','day_Open_low_per', 
                 'Low_Open_diff', 'ADR_percent_5d', 'General__Sector', 'SharesStats__SharesFloat']].sort_values(
                by=['Date', 'price_cat','day_Open_low_per'], ascending=[True, True, True]).reset_index(drop=True)

# 'ADR_percent_5d': ['min','median','mean','max'], 

(16, 382)


,Date,price_cat,Ticker,Adj_Close,day_Open_low_per,Low_Open_diff,ADR_percent_5d,General__Sector,SharesStats__SharesFloat
0,2023-08-09,GT_20,UPST,34.029999,-22.043013,-9.430,17.599001,Financial Services,70.862272
1,2023-08-09,GT_20,RVNC,20.219999,-13.000000,-2.860,7.617000,Healthcare,75.846418
2,2023-08-09,GT_20,CVNA,41.439999,-12.839402,-6.100,10.946000,Consumer Cyclical,75.194391
3,2023-08-09,GT_20,RBLX,29.459999,-10.388410,-3.370,4.637000,Communication Services,469.350999
4,2023-08-09,GT_2_LT_5,RYAM,3.270000,-25.382652,-0.995,11.299000,Basic Materials,55.471866
5,2023-08-09,GT_2_LT_5,KIND,2.460000,-21.180555,-0.610,8.186000,Communication Services,131.910326
6,2023-08-09,GT_2_LT_5,TUP,4.240000,-16.770187,-0.810,24.389999,Consumer Cyclical,38.804000
7,2023-08-09,GT_2_LT_5,GRWG,2.800000,-16.716419,-0.560,8.816000,Consumer Cyclical,58.144759
8,2023-08-09,GT_5_LT_20,EBS,5.920000,-21.448864,-1.510,9.186000,Healthcare,45.298195
9,2023-08-09,GT_5_LT_20,NVEI,18.129999,-17.336449,-3.710,7.161000,Technology,62.620138


In [55]:
#%reset -f 